# `s2Dcd` basic example
This is a quite simple example to illustrate how to perform a 3D simulation by using 2D training images only with the module `s2Dcd` in practice.

You can find a Python source version of this code in the same folder of this notebook (or export is as you wish from a Jupyter notebook).




In [19]:
# Import "standard" modules
import os
import numpy as np
import geone as gn
import json
import copy
import matplotlib.pylab as pl

Import some modules contained in the `s2Dcd` package.

In [20]:
# Import non-standard modules
import s2Dcd.s2Dcd as s2Dcd
import s2Dcd.utili as utili

In [21]:
# Print header and start counting time
# (this is useful only for a standalone run, not in a Jupyter notebook...)
time_start = utili.print_start() 

# If you can use a parallel version of the MPS simulation core,
# define here the number of threads.
nthreads = 8



    ***********************
    ***  1059164934.py  ***
    ***********************
    START:  Wed Aug 17 11:03:17 2022


## Read the info of the *pseudo* 3D training image
Then, the next step is to read all the information related to the training images (TIs) used. For the moment, we load the information as we would have when working with a complete 3D TI (that we don't have here, actually).
This information is contained in the `JSON` file `ti3Ddict.json`, which contains all the information needed to define the geometry of a 3D TI. See the module (geone)[https://github.com/randlab/geone] for more info about the image format.

In our case (see later in the script), we are using two (pseudo) 3D TIs:
- one along the plane $xz$, of dimensions $250{\times}1{\times}250$
- one along the plane $xy$, of dimensions $250{\times}250{\times}1$

Here no training image is provided for the plane $yz$, but if for your case study you have one, you can use it...

Therefore, ideally, the two aforementioned TIs could be two slices of a full 3D TI, of dimensions $250{\times}250{\times}250$. This is the information contained in the file `ti3Ddict.json`.


In [22]:
# %% Read some info about the TI

# Read from an external file the dimensions that would have a 3D TI
# (when composed by the 2D slices considered here)
ti3Ddict_file = "ti3Ddict.json"
with open(ti3Ddict_file, "r") as json_in:
    ti3Ddict = json.load(json_in)

# Use the parameters to create an empty 3D TI
ti3Ddict["val"] = np.full([ti3Ddict["nx"], ti3Ddict["ny"], ti3Ddict["nz"]],
                          np.nan)

With the lines above, the content of the `JSON` file is directly translated into a Python dictionary, that once properly initializated (for example with *NaN* values), can be directly used to create an `Image` object within `geone`:

In [23]:
ti3D = gn.img.Img(**ti3Ddict)

## Create a *template* for the simulation parameters of the simulation steps
The next step is to read all the information related to the simulation grid and the simulation parameters.
This information is contained into the `JSON` file `ds3Ddict.json`.

In [24]:
# %% Read the simulation grid and the main simulation parameters    
#

# Create a dictionary containing the parameters that will be used to set up
# the default input parameters of the DeeSse
ds3Ddict_file = "ds3Ddict.json"
with open(ds3Ddict_file, "r") as json_in:
    ds3Din_dict = json.load(json_in)

Just to lighten the notation, use some variables...

In [25]:
nx = ds3Din_dict["nx"]    
ny = ds3Din_dict["ny"]
nz = ds3Din_dict["nz"]
sx = ds3Din_dict["sx"]
sy = ds3Din_dict["sy"]
sz = ds3Din_dict["sz"]
ox = ds3Din_dict["ox"]
oy = ds3Din_dict["oy"]
oz = ds3Din_dict["oz"]
nv = ds3Din_dict["nv"]
varname = ds3Din_dict["varname"]
name = "res3D.gslib"
# Add a 3D TI to the parameter file
ds3Din_dict["TI"] = np.array([ti3D])

From the information about the simulation grid retrieved above, we create an empty simulation grid initialized with *NaN*. This grid will be used as container for the results of the sequence of 2D simulation steps.

Here we do not provide conditioning data, but if you have some you can load in the proper location of this 3D container as they will be taken into account in the next steps of the alogoritm.

In [26]:
# %% Create an empty simulation grid to be filled with the sequential 2D simulations

# Create an empty simulation grid (Img) to be filled with the 2D simulations
val = np.full([ds3Din_dict["nx"], ds3Din_dict["ny"], ds3Din_dict["nz"]], np.nan)
res3D = gn.img.Img(nx, ny, nz, sx, sy, sz, ox, oy, oz, nv, val, varname, name)

With the *DeeSse* input parameters set un in the dictionary ``ds3Din_dict`` we now instanciate a template for the parameters to be used in each step of the simulation sequence.

In [27]:
# %% Set up the parameters and the training images

# Set up the general parameters to be used for the target 3D final result
ds3Din = gn.deesseinterface.DeesseInput(**ds3Din_dict)

## Set up the simulation parameters for the different simulation planes

Now it is possible to set up the information related to each simulation step along the planes $xy$, $xz$ and $yz$.
In this example, a TI along the plane $yz$ is not provided. Therefore, we set the parameter set for this plane to ``None``

In [28]:
# Set up the main parameters for the simulation along the plane *yz*
# from the 3D template (along this direction we do not have a TI in this example)
ds_yz_in = None

Differently, for the two other planes $xz$ and $xy$ (or directions perpendicular to $x$ and $z$ axes) we set up the simulation parameters for the corresponding simulation steps.
The steps required are:
1. make a `copy` of the simulation parameter template.
2. set the size of the simulation grid perpendicular to the 2D plane to `1`
3. read the TI to be used along that plane, and add it to the parameters created at step 1.

In [29]:
# Set up the main parameters for the simulation along the plane *xz*, starting
# from the 3D template
ds_xz_in = copy.deepcopy(ds3Din)
# For the moment we only set up the size of the SG. The origin will be defined
# within the simulation sequence.
ds_xz_in.ny = 1

# Read the training image along the plane xz
ti_xz = gn.img.readImageGslib(os.path.join("..","data","strebelle", 
                                           "ti_250x1x250.gslib"))
ds_xz_in.TI = np.array([ti_xz])

# Set up the main parameters for the simulation along the plane *xy*, starting
# from the 3D template
ds_xy_in = copy.deepcopy(ds3Din)
ds_xy_in.nz = 1
# Read the training image along the plane *xy*
ti_xy = gn.img.readImageGslib(os.path.join("..","data","strebelle", 
                                           "ti_250x250x1.gslib"))
ds_xy_in.TI = np.array([ti_xy])

## Setting of other simulation parameters

Now it is time to set up some final parameters.

> **NOTE**
> The variable `step_max` is quite useful if you want to quickly test if your settings are OK without waiting until the simulation of all the nodes contained in the 3D simulation grid is completed.
> 
> For example, you can set it to a low value, like for example `5`, and then you can verify if your simulation settings were OK by looking at the results. You can for example set the arguments `print_hd` and `print_sim` of the function `create_seq` to `True` (see the next cells) and then the hard data used in the intermediate simulation steps of the sequence and the intermediate simulation results will be saved as *VTK* files.
>
> If you don't need to debug of check, then you can set `step_max` to a very big integer, and the simulation will be stopped in any case when the 3D simulation grid is filled.

In [30]:
# %% perform the simulation

# The max number of simulation steps to be performed. You can use it
# if you want to stop the simulation before the 3D domain is
# completed (i.e. for quick testing purposes...).
step_max = 1000000

It is also useful to print some info about the simulation settings...

In [31]:
# Print some simulation info
s2Dcd.print_sim_info(ds3Din, ds_yz_in, ds_xz_in, ds_xy_in, nthreads)


    Dimension of the simulation domain: (52x71x64)
    Plane normal to axis *x*:
        No TI defined 
    Plane normal to axis *y*:
        MPS simulation engine: "DeeSse" with 8 threads
        TI file:               "../data/strebelle/ti_250x1x250.gslib"
    Plane normal to axis *z*:
        MPS simulation engine: "DeeSse" with 8 threads
        TI file:               "../data/strebelle/ti_250x250x1.gslib"


## Create the simulation sequence

The core of the procedure is the creation of the simulation sequence with the function `create_seq`. The simulation sequence contains, for each step, the training image to be used, the simulation parameters, and the coordinate along the direction perpedicular to the simulation plane where the simulation is performed.

> **NOTE** By default, the `s2Dcd` will create a sequence that, in a case where the size of the simulation grid along the three dimensions are comparable, optimizes the number of intersections and therefore the "coherence" of the 2D slices (see the function `s2Dcd.matrioska_interval` for more details).
However, depending on the shape of your domain or on the positions of your conditioning data, you can also customize the sequence as you wish.

As anticipated before, if you need to check if the simulation procedure is working properly you can set the arguments `print_hd` and `print_sim` of the function `create_seq` to `True`.

The fuction should warn you one TI is missing along some plane. If you expect that, don't worry about the warning message...

In [32]:
# Create the simulation sequence
seq = s2Dcd.create_seq(ds3Din, ds_yz_in, ds_xz_in, ds_xy_in, nthreads)

Then run the simulation and print some useful information...

In [33]:
#
# Simulation
#
s2Dcd.sim_run(seq, step_max, res3D, ds3Din, nthreads)


    *** Simulation starts *** 
    Seq.step   1 of 135 (max) - slice simulated: y =    0
* checking out license OK.
    Seq.step   2 of 135 (max) - slice simulated: z =    0
* checking out license OK.
    Seq.step   3 of 135 (max) - slice simulated: y =   70
* checking out license OK.
    Seq.step   4 of 135 (max) - slice simulated: z =   63
* checking out license OK.
    Seq.step   5 of 135 (max) - slice simulated: y =   35
* checking out license OK.
    Seq.step   6 of 135 (max) - slice simulated: z =   31
* checking out license OK.
    Seq.step   7 of 135 (max) - slice simulated: y =   17
* checking out license OK.
    Seq.step   8 of 135 (max) - slice simulated: z =   15
* checking out license OK.
    Seq.step   9 of 135 (max) - slice simulated: y =   53
* checking out license OK.
    Seq.step  10 of 135 (max) - slice simulated: z =   47
* checking out license OK.
    Seq.step  11 of 135 (max) - slice simulated: y =    8
* checking out license OK.
    Seq.step  12 of 135 (max) - s

* checking out license OK.
    Seq.step  98 of 135 (max) - slice simulated: z =   32
* checking out license OK.
    Seq.step  99 of 135 (max) - slice simulated: y =   37
* checking out license OK.
    Seq.step 100 of 135 (max) - slice simulated: z =   34
* checking out license OK.
    Seq.step 101 of 135 (max) - slice simulated: y =   39
* checking out license OK.
    Seq.step 102 of 135 (max) - slice simulated: z =   36
* checking out license OK.
    Seq.step 103 of 135 (max) - slice simulated: y =   41
* checking out license OK.
    Seq.step 104 of 135 (max) - slice simulated: z =   38
* checking out license OK.
    Seq.step 105 of 135 (max) - slice simulated: y =   43
* checking out license OK.
    Seq.step 106 of 135 (max) - slice simulated: z =   40
* checking out license OK.
    Seq.step 107 of 135 (max) - slice simulated: y =   46
* checking out license OK.
    Seq.step 108 of 135 (max) - slice simulated: z =   42
* checking out license OK.
    Seq.step 109 of 135 (max) - slice 

In [34]:
# Stop counting time
utili.print_stop(time_start)


    STOP:   Wed Aug 17 11:03:24 2022
    Ellapsed time [sec]: 7.45




Finally, save the results as a *VTK* file and draw it:

In [35]:
gn.img.writeImageVtk(res3D, "res3D.vtk", missing_value=-9999999)

# %% Print the result in 3D with PyVista
pl.figure()
gn.imgplot3d.drawImage3D_surface(res3D, text='TI', scalar_bar_kwargs={'vertical':True})
pl.show()


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

<Figure size 432x288 with 0 Axes>